<a href="https://colab.research.google.com/github/bobthesiddharth/Vehicle-Cut-in-Detection/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    line_img = np.zeros(
        (
            img.shape[0],
            img.shape[1],
            3
        ),
        dtype=np.uint8
    )
    img = np.copy(img)
    if lines is None:
        return img
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    return img

def detect_cars(image):
    # Update the path to the correct location where the haarcascade_car.xml is stored
    car_cascade = cv2.CascadeClassifier('haarcascade_car.xml')  # Update this path
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cars = car_cascade.detectMultiScale(gray, 1.1, 1)
    for (x, y, w, h) in cars:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
    return image

def pipeline(image):
    """
    An image processing pipeline which will output
    an image with the lane lines and cars annotated.
    """
    height = image.shape[0]
    width = image.shape[1]
    region_of_interest_vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    cannyed_image = cv2.Canny(gray_image, 100, 200)

    cropped_image = region_of_interest(
        cannyed_image,
        np.array(
            [region_of_interest_vertices],
            np.int32
        ),
    )

    lines = cv2.HoughLinesP(
        cropped_image,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        lines=np.array([]),
        minLineLength=40,
        maxLineGap=25
    )

    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = (y2 - y1) / (x2 - x1)
                if math.fabs(slope) < 0.5:
                    continue
                if slope <= 0:
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else:
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])

    if left_line_x and left_line_y and right_line_x and right_line_y:
        min_y = int(image.shape[0] * (3 / 5))
        max_y = int(image.shape[0])

        # Fit polynomial to x and y values for left and right lines
        if left_line_x and left_line_y:
            poly_left = np.poly1d(np.polyfit(
                left_line_y,
                left_line_x,
                deg=1
            ))
            left_x_start = int(poly_left(max_y))
            left_x_end = int(poly_left(min_y))
        else:
            left_x_start = left_x_end = 0

        if right_line_x and right_line_y:
            poly_right = np.poly1d(np.polyfit(
                right_line_y,
                right_line_x,
                deg=1
            ))
            right_x_start = int(poly_right(max_y))
            right_x_end = int(poly_right(min_y))
        else:
            right_x_start = right_x_end = 0

        # Draw the left and right lines on the image
        line_image = draw_lines(
            image,
            [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
            ]],
            thickness=5,
        )
    else:
        line_image = image  # If no lines are detected, return the original image

    # Detect cars and draw red rectangles around them
    final_image = detect_cars(line_image)

    return final_image

# Example usage
if __name__ == "__main__":
    white_output = 'solidWhiteRight_output.mp4'
    clip1 = VideoFileClip("/content/Lane.mp4")
    white_clip = clip1.fl_image(pipeline)
    white_clip.write_videofile(white_output, audio=False)


Moviepy - Building video solidWhiteRight_output.mp4.
Moviepy - Writing video solidWhiteRight_output.mp4



t:  29%|██▉       | 530/1801 [02:42<09:58,  2.12it/s, now=None]WARNING:py.warnings:<ipython-input-16-d0956d82bc08>:83: RuntimeWarning: divide by zero encountered in scalar divide
  slope = (y2 - y1) / (x2 - x1)



Moviepy - Done !
Moviepy - video ready solidWhiteRight_output.mp4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')